# Data Cleaning

In this notebook we'll go through the process of cleaning with the `orders` , `orderlines` and `products` datasets.

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None) # to increase the width of the columns

In [3]:
# Load the datasets from Google Drive
# # orders.csv
# url = "https://drive.google.com/file/d/1d0ATmRAa5z24ai0UkNgGVxvQ17_FNLoo/view?usp=sharing"
# path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
# orders = pd.read_csv(path)
#
#
# # orderlines.csv
# url = "https://drive.google.com/file/d/1GKErSdkADdYKIgeuDMdLBvNubip6Amoy/view?usp=sharing"
# path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
# orderlines = pd.read_csv(path)
#
# # products.csv
# url = "https://drive.google.com/file/d/1y1XBUo2ZKTvc5uymVOHUxM5SsrrJF1yN/view?usp=sharing"
# path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
# products = pd.read_csv(path)

#-----------------------------
# Load the datasets from Local
path = "../data/raw-data/orders.csv"
orders = pd.read_csv(path)

path = "../data/raw-data/orderlines.csv"
orderlines = pd.read_csv(path)

path = "../data/raw-data/products.csv"
products = pd.read_csv(path)

Before we begin, let's create a copy of the `orders` and `orderlines`  and `products` DataFrames. This way we are sure any of our changes won't affect the original DataFrames.

In [4]:
orders_df = orders.copy()

In [5]:
orderlines_df = orderlines.copy()
products_df = products.copy()

we begin data cleaning is by exploring using `.info()`. This will tell us:
* The shape of the DataFrame
* The names of the columns
* If there are any missing values
* The datatypes of the columns

By exploring the missing values and correcting any incorrect datatypes, we often come across inconsistencies in our data.

Beyond this, we should also have a **check for any duplicate rows**.

## 1.&nbsp; Duplicates
We can check for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method.

We can then delete these rows, if we wish, using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [6]:
# orders
orders_df.duplicated().sum()

np.int64(0)

In [7]:
# orderlines
orderlines_df.duplicated().sum()

np.int64(0)

In [8]:
# products
products_df.duplicated().sum()

np.int64(8746)

In [9]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [10]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


* `date` should be a datetime datatype
* `unit_price` should be a float datatype

In [11]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


## 3.&nbsp; Missing values

### 3.1.&nbsp; Orders
* `total_paid` has 5 missing values

In [12]:
print(f"5 missing values represents {((orders_df.total_paid.isna().sum() / orders_df.shape[0])*100).round(5)}% of the rows in our DataFrame")

5 missing values represents 0.0022% of the rows in our DataFrame


A quick way to find out a percentage here, if you don't need to print out a sentence for yourself/students/colleagues is `.value_count(normalize=True)`

In [13]:
orders_df.total_paid.isna().value_counts(normalize=True)

total_paid
False    0.999978
True     0.000022
Name: proportion, dtype: float64

As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [14]:
orders_df = orders_df.loc[~orders.total_paid.isna(), :]

Should we have a significant number of missing values in the future, we have a choice:
+ we can impute the values
+ we can take the values from other DataFrames, if they are present there
+ we can delete the values
+ or any number of other creative solutions

### 3.2.&nbsp; Orderlines
There are no missing values in `orderlines`

## 4.&nbsp; Datatypes

### 4.1.&nbsp; Orders
* `created_date` should become datetime datatype

In [15]:
orders_df["created_date"] = pd.to_datetime(orders_df["created_date"])

### 4.1.&nbsp; Orderlines
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 4.1.1.&nbsp; `date`

In [16]:
orderlines_df["date"] = pd.to_datetime(orderlines_df["date"])

#### 4.1.2.&nbsp;`unit_price`

In [17]:
#orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

As you can see when we try to convert `unit_price` to a numerical datatype, we receive a `ValueError` telling us that pandas doesn't understand the number `1.137.99`. This is probably because numbers cannot have 2 decimal points. Let's see if there are any other numbers like this.

In [18]:
orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()

unit_price
False    257814
True      36169
Name: count, dtype: int64

Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [19]:
two_dot_percentage = ((orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts().iloc[1] / orderlines_df.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 12.3% of the rows in our DataFrame


This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows, and I'd like some quick, accurate results to show. If we have time at the end, we can come back and investigate this problem further, maybe there's a solution?

Each row of `orderlines` represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in `orderlines`, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.

We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [20]:
two_dot_order_ids_list = orderlines_df.loc[orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]

orderlines_df = orderlines_df.loc[~orderlines_df.id_order.isin(two_dot_order_ids_list)]

In [21]:
orderlines_df.shape[0]

216250

We still have 216250 rows in orderlines to work with. This should be more than enough for our evaluation.

Now that all of the 2 decimal point prices have been removed, let's try again to convert the column `unit_price` to the correct datatype.

In [22]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

It worked perfectly

# Clean the `products` DataFrame
We begin here to clean the products DataFrame.

In [23]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


We'll go through the steps above in order
* Duplicates
* Missing values
* Datatypes

But I think we can all see straight away from `products.head()` above that some of the prices in `promo_price` look wrong. Let's make sure we deal with this later.

## Duplicates

In [24]:
products_df.duplicated().sum()

np.int64(8746)

Wow, that's a lot of duplicates. Let's get rid of them.

In [25]:
products_df = products_df.drop_duplicates()

In [26]:
products_df

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pro (2006/07/08).,34.99,31.99,1,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horizontally for Apple Watch,29.99,269.903,1,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartments MacBook up to 13 inches up to 10 inches Cases,69.95,649.903,1,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartments MacBook up to 13 inches up to 10 inches Cases,69.95,649.903,1,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartments MacBook up to 13 inches up to 10 inches Cases,69.95,649.903,0,1392


## `.info()`

In [27]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10573 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  int64 
 6   type         10530 non-null  object
dtypes: int64(1), object(6)
memory usage: 661.2+ KB


### Missing values
We can see from `.info()` above that we have missing values in `desc` and `price`

#### `desc`

In [28]:
products_df["desc"].isna().sum()

np.int64(7)

7 is a very small number to have missing, let's have a closer look

In [29]:
products_df.loc[products_df['desc'].isna(), :]

,sku,name,desc,price,promo_price,in_stock,type
16126,WDT0211-A,"Open - Purple 2TB WD 35 ""PC Security Mac hard drive and NAS",NaN,107,814.659,0,1298
16128,APP1622-A,"Open - Apple Smart Keyboard Pro Keyboard Folio iPad 9.7 """,NaN,1.568.206,1.568.206,0,1298
17843,PAC2334,Synology DS718 + NAS Server | 10GB RAM,NaN,566.35,5.659.896,0,12175397
18152,KAN0034-A,"Open - Kanex USB-C Gigabit Ethernet Adapter MacBook 12 """,NaN,29.99,237.925,0,1298
18490,HTE0025,Hyper Pearl 1600mAh battery Mini USB Mirror and Comic Blond,NaN,24.99,22.99,1,1515
18612,OTT0200,OtterBox External Battery Power Pack 20000 mAHr,NaN,79.99,56.99,1,1515
18690,HOW0001-A,Open - Honeywell thermostat Lyric zonificador T6 Intelligent Wireless (cable),NaN,199.99,1.441.174,0,11905404


We have 2 choices here:
* We can quickly and easily remove these rows.
* Or, alternatively, the products names here are quite descriptive, so I'm tempted to just copy them to the description column, so that there is a description if we later want utilise this column. I wouldn't recommend this if this DataFrame was the source of truth for our website. But this is not the case here, and we're not faking any information (guessing a price or so), so I'm happy with this option

In [30]:
products_df.loc[products_df['desc'].isna(), 'desc'] = products_df.loc[products_df['desc'].isna(), 'name']

In [31]:
products_df.loc[products_df['desc'].isna(), :]

,sku,name,desc,price,promo_price,in_stock,type


Did you also notice above that we have the dreaded two decimal point problem in both the `price` and `promo_price` columns? We can also see prices with 3 decimal places, prices should have 2 decimal places: this gives us more cause for concern

#### `price`

In [32]:
products_df.price.isna().sum()

np.int64(46)

In [33]:
print(f"The missing values in price are {(products_df.price.isna().value_counts(normalize=True).iloc[1] * 100).round(2)}% of all rows in the DataFrame")

The missing values in price are 0.43% of all rows in the DataFrame


Let's simply delete these rows to ensure that we can trust the numbers in our final DataFrame. Afterall, the price is very important when investigating discounts.

Option 1: `.loc`

In [34]:
products_df = products_df.loc[~products['price'].isna()]

Option 2: `.dropna()`

In [35]:
#products_df = products_df.dropna(subset=['price'])

#### `type`

In [36]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10534 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10534 non-null  object
 1   name         10534 non-null  object
 2   desc         10534 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10534 non-null  object
 5   in_stock     10534 non-null  int64 
 6   type         10484 non-null  object
dtypes: int64(1), object(6)
memory usage: 658.4+ KB


Type isn’t an essential piece of data for the analysis and is therefore allowed to carry missing values.
The only place it comes in later is as an optional route to category creation, where someone might still choose to drop the rows with missing values, however one can still use name and desc to categorize those rows.


### Data types

We saw from looking at the output of `.info()` that both `price` and `promo_price` have been stored as objects and not as a numerical datatypes. We also saw while solving other problems that both columns have some prices with 3 decimal places and others with 2 decimal points - the latter will prevent us from converting the datatype to numerical, so first we must investigate and solve these problems.

#### `price`

First, let's see how many values are affected by the 2-decimal-dot problems or 3 decimal places.

In [37]:
price_problems_number = products_df.loc[(products_df.price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products_df.price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
price_problems_number

542

In [38]:
print(f"The column price has in total {price_problems_number} wrong values. This is {round(((price_problems_number / products_df.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column price has in total 542 wrong values. This is 5.15% of the rows of the DataFrame


5.15% is a reasonable amount of our data. However, the price column will be important to understanding discounts, so I'd like it to be very trustworthy as we are basing business decisions on it. Therefore, we'll delete these rows

In [39]:
products_df = products_df.loc[(~products_df.price.astype(str).str.contains("\d+\.\d+\.\d+"))&(~products_df.price.astype(str).str.contains("\d+\.\d{3,}")), :]

In [40]:
products_df

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pro (2006/07/08).,34.99,31.99,1,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horizontally for Apple Watch,29.99,269.903,1,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartments MacBook up to 13 inches up to 10 inches Cases,69.95,649.903,1,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartments MacBook up to 13 inches up to 10 inches Cases,69.95,649.903,1,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartments MacBook up to 13 inches up to 10 inches Cases,69.95,649.903,0,1392


To complete our task, let's convert the column to a numeric datatype

In [41]:
products_df["price"] = pd.to_numeric(products_df["price"])

In [42]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9992 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          9992 non-null   object 
 1   name         9992 non-null   object 
 2   desc         9992 non-null   object 
 3   price        9992 non-null   float64
 4   promo_price  9992 non-null   object 
 5   in_stock     9992 non-null   int64  
 6   type         9946 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 624.5+ KB


#### `promo_price`

Again, let's begin by seeing how many values are affected by the 2-decimal-dots problem, or the 3 decimal-places problem

In [43]:
promo_problems_number = products_df.loc[(products_df.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products_df.promo_price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
promo_problems_number

9232

In [44]:
print(f"The column promo_price has in total {promo_problems_number} wrong values. This is {round(((promo_problems_number / products_df.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column promo_price has in total 9232 wrong values. This is 92.39% of the rows of the DataFrame


WOW!!! That's a lot of wrong data. Let's have a quick investigate to check that's correct. We'll make a DataFrame by copy-pasting the code we used above and then look at a large sample to check that all the numbers in the `promo_price` column really have either 2 decimal points or 3 decimal places.

In [45]:
promo_price_df = products_df.loc[(products_df.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products_df.promo_price.astype(str).str.contains("\d+\.\d{3,}")), :]
promo_price_df.sample(50)

,sku,name,desc,price,promo_price,in_stock,type
517,SEV0012,HDD or SSD installation service MacBook / Pro,HDD or SSD installation on your MacBook / Pro + Data Transfer.,49.99,499.851,0,20642062
2258,APP0984,"Apple MacBook 12 ""Core M 12 Ghz | 8GB RAM | 512GB Flash Gold",New MacBook Retina 12-inch screen 8GB RAM 512GB 11GHz Flash.,1799.00,17.449.894,0,1282
18714,WAC0158-A,Wacom Cintiq Ergo Stand,Reconditioned professional support for Wacom Cintiq 27QHD,429.90,3.424.682,0,101781405
16246,APP2234,"Apple iMac 27 ""Core i5 Retina 5K 38GHz | 8GB | 1TB SSD",IMac desktop computer 27 inch 5K Retina 8GB RAM 1TB SSD PCle,3319.00,31.200.044,0,"5,74E+15"
2765,ACM0035,"Acme Skinny Sleeve Case MacBook Air 11 ""Lilac / Turquoise",Neoprene sleeve for MacBook Air 11 inch.,34.95,269.891,0,13835403
9940,PAC0978,"Apple iMac 27 ""Core i5 3.2GHz Retina 5K | 32GB | 512GB Flash | AMD Radeon R9 M390",IMac desktop computer 27 inch 5K Retina i5 3.2GHz 512GB Flash RAM 32GB and AMD Radeon R9 M390 (MK472Y / A).,3409.00,28.569.903,0,"5,74E+15"
15580,SXA0018,"SecurityXtra Security Support iMac 215 """,Security support with attachment to the desktop 215 inch iMac,109.99,890.003,0,8696
1577,PAC0590,Synology DS415 + Pack | WD 24TB Network,Synology DS415 + Pack of 24TB WD Network for Mac and PC.,1770.99,15.389.905,0,12175397
13786,SNN0040-A,(Open) Sonnet SuperSpeed ​​USB 3.0 ExpressCard / 34,SuperSpeed ​​USB 3.0 PCI adapter ExpressCard / 34 for Mac.,74.90,458.333,0,1298
18879,MOP0111,Mophie Juice Pack Air Battery Case for iPhone 2.420mAh 8 Plus / 7 Red Plus,Mophie Battery Case for iPhone 2.420mAh 8 Plus or 7-Plus,99.95,299.899,1,11865403


So we were correct, over 90% of the data in this column is corrupt. There's no point deleting all of these rows, then we would barely have a products table. Instead, as it's only this column that appears to be very untrustworthy, we will delete the column.

In [46]:
products_cl = products_df.drop(columns=["promo_price"])

In [47]:
products_cl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9992 entries, 0 to 19325
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       9992 non-null   object 
 1   name      9992 non-null   object 
 2   desc      9992 non-null   object 
 3   price     9992 non-null   float64
 4   in_stock  9992 non-null   int64  
 5   type      9946 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 546.4+ KB


Obviously, there's now no need to convert `promo_price` to a numerical datatype

to download/save your new DataFrames. Also, give them an obvious name, so that you know they are the cleaned version and not the original DataFrame.

In [48]:
#from google.colab import files:

#orders_df.to_csv("orders_cl.csv", index=False)
#files.download("orders_cl.csv")

#orderlines_df.to_csv("orderlines_cl.csv", index=False)
#files.download("orderlines_cl.csv")

#products_cl.to_csv("products_cl.csv", index=False)
#files.download("products_cl.csv")

#------------------
#to save locally:
orders_df.to_csv("../data/cleaned-data/orders_cl.csv")
orderlines_df.to_csv("../data/cleaned-data/orderlines_cl.csv")
products_cl.to_csv("../data/cleaned-data/products_cl.csv")
